In [13]:
import common

import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from dotenv import load_dotenv
from datetime import datetime
from zoneinfo import ZoneInfo
import wandb

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
is_soft_voting = True
augment_ratio = 1

In [16]:
def get_all_targets_count():
    sample_submission_df = pd.read_csv("datasets_fin/sample_submission.csv")
    return len(sample_submission_df)
    
def hard_voting(predictions):
    predictions = np.asarray(predictions)
    return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

def soft_voting(predictions):
    predictions = np.asarray(predictions)
    mean_axis0 = np.mean(predictions, axis=0)
    
    # 증강된 데이터에 대한 예측값도 고려하기.
    all_targets_count = get_all_targets_count()
    
    aug_size = len(mean_axis0) / all_targets_count
    assert len(mean_axis0) % all_targets_count == 0
    aug_size = int(aug_size)
    
    if aug_size > 1:
        bulk_list = []
        step = 0
        
        for i in range(0, aug_size):
            bulk_list.append(mean_axis0[step:step + all_targets_count])
            step += all_targets_count
        
        bulk_list = np.asarray(bulk_list)
        mean_axis0 = np.mean(bulk_list, axis=0)
        
    return mean_axis0.argmax(axis=1)

In [17]:
fold_1_file_list = [
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_1_aug_200_vl_0.2419_va_0.9389_vf1_0.9370_fold_1_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_2_aug_200_vl_0.2965_va_0.9446_vf1_0.9441_fold_1_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_3_aug_200_vl_0.5071_va_0.9146_vf1_0.9181_fold_1_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_4_aug_200_vl_0.2366_va_0.9400_vf1_0.9404_fold_1_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_5_aug_200_vl_0.3815_va_0.9400_vf1_0.9387_fold_1_folds_2.pt', 
]

fold_2_file_list = [
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_1_aug_200_vl_0.3152_va_0.9308_vf1_0.9284_fold_2_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_2_aug_200_vl_0.2380_va_0.9481_vf1_0.9488_fold_2_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_3_aug_200_vl_0.2916_va_0.9423_vf1_0.9407_fold_2_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_4_aug_200_vl_0.2985_va_0.9412_vf1_0.9405_fold_2_folds_2.pt', 
    'model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_5_aug_200_vl_0.3988_va_0.9377_vf1_0.9383_fold_2_folds_2.pt', 
]

In [18]:
cnt = 0

for fold_1_filename in fold_1_file_list:
    for fold_2_filename in fold_2_file_list:
        print(f'fold 1 filename: {fold_1_filename}')
        print(f'fold 2 filename: {fold_2_filename}')
        
        model_checkpoint_filename_list = [
            fold_1_filename,
            fold_2_filename,
        ]

        model_list = []
        tst_loader_list = []

        for cp_filename in model_checkpoint_filename_list:
            checkpoint = torch.load(cp_filename, map_location = device)
            
            print(checkpoint['model'], checkpoint['tst_img_size'], checkpoint['batch_size'])
            
            model = timm.create_model(
                checkpoint['model'],
                pretrained = True,
                num_classes = 17,
            ).to(device)
            
            model.load_state_dict(checkpoint['model_state_dict'])    
            model_list.append(model)
            
            ##
            tst_transform = common.create_tst_transform(checkpoint['tst_img_size'])
            tst_aug_transform = common.create_trn_aug_transform(checkpoint['tst_img_size'])

            # tst_dataset = common.ImageDataset(
            #     "datasets_fin/sample_submission.csv",
            #     "datasets_fin/test/",
            #     transform = tst_transform
            # )
            
            tst_dataset = common.ImageDataset(
                "datasets_fin/sample_submission.csv",
                "datasets_fin/test/",
                transform=tst_transform, 
                aug_transform=tst_aug_transform, 
                augment_ratio=augment_ratio)

            tst_loader = DataLoader(
                tst_dataset,
                batch_size = checkpoint['batch_size'],
                shuffle = False,
                num_workers = 12,
                pin_memory = True
            )
            
            tst_loader_list.append(tst_loader)

        print(len(model_list), len(tst_loader_list))
        
        preds_list = []

        for model, tst_loader in zip(model_list, tst_loader_list):
            preds = common.get_preds_list_by_tst_loader(model, tst_loader, device, is_soft_voting)
            preds_list.append(preds)
        
        # 최종 예측
        if is_soft_voting:
            final_pred = soft_voting(preds_list)
        else:
            final_pred = hard_voting(preds_list)

        # csv 로 저장
        common.preds_list_to_save_to_csv(final_pred, tst_loader, f'pred_0808_hoho_{cnt}.csv')
        
        cnt += 1

fold 1 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_1_aug_200_vl_0.2419_va_0.9389_vf1_0.9370_fold_1_folds_2.pt
fold 2 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_1_aug_200_vl_0.3152_va_0.9308_vf1_0.9284_fold_2_folds_2.pt
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
2 2


100%|██████████| 99/99 [00:11<00:00,  8.47it/s]


fold 1 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_1_aug_200_vl_0.2419_va_0.9389_vf1_0.9370_fold_1_folds_2.pt
fold 2 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_2_aug_200_vl_0.2380_va_0.9481_vf1_0.9488_fold_2_folds_2.pt
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
2 2


100%|██████████| 99/99 [00:11<00:00,  8.58it/s]


fold 1 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_1_aug_200_vl_0.2419_va_0.9389_vf1_0.9370_fold_1_folds_2.pt
fold 2 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_3_aug_200_vl_0.2916_va_0.9423_vf1_0.9407_fold_2_folds_2.pt
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
2 2


...

100%|██████████| 99/99 [00:11<00:00,  8.51it/s]


fold 1 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_5_aug_200_vl_0.3815_va_0.9400_vf1_0.9387_fold_1_folds_2.pt
fold 2 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_4_aug_200_vl_0.2985_va_0.9412_vf1_0.9405_fold_2_folds_2.pt
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
2 2


100%|██████████| 99/99 [00:11<00:00,  8.46it/s]


fold 1 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_5_aug_200_vl_0.3815_va_0.9400_vf1_0.9387_fold_1_folds_2.pt
fold 2 filename: model_bak2/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_5_aug_200_vl_0.3988_va_0.9377_vf1_0.9383_fold_2_folds_2.pt
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32
2 2


100%|██████████| 99/99 [00:11<00:00,  8.46it/s]
